# Minecraft Enchantment List Generator

In [2]:
from urllib.request import urlretrieve
from os import path

import pandas as pd
from bs4 import BeautifulSoup as Soup

FILE = path.join("/tmp", "enchantment_list_pc.html")
OUTFILE = "enchantments.csv"
URL = "https://www.digminecraft.com/lists/enchantment_list_pc.php"

if not path.isfile(FILE):
    urlretrieve(URL, FILE)

In [3]:
def enchants_permitted():
    with open(FILE) as file:
        soup = Soup(file, "html.parser")
    
    table = soup.find("table", {"id": "minecraft_items"})
    permitted = []
    unwanted = (
        ".png",
        "_sm",
        "iron_",
        "enchanted_",
    )

    for img in table.find_all("img"):
        item_types = img["data-src"].split("/")[-1]
        if "fishing_rod" in item_types:
            item_types = item_types.replace("fishing_rod", "fishingrod")
        
        for s in unwanted:
            if s in item_types:
                item_types = item_types.replace(s, "")
        
        item_types = item_types.split("_")
        item_types = ["fishing_rod" if item == "fishingrod" else item for item in item_types]
        permitted.append(" ".join(item_types))

    return permitted

In [4]:
raw_data = pd.read_html(FILE)[0]
names = raw_data["Enchantment(Minecraft ID Name)"].apply(lambda name: name.split("(")[0])
id_names = raw_data["Enchantment(Minecraft ID Name)"].apply(lambda name: name.split("(")[1].replace(")", ""))
data = pd.DataFrame(
    dict(
        id_name = id_names,
        name = names,
        max_level = raw_data["Max Level"],
        description = raw_data["Description"],
        items = raw_data["Items"],
    )
)
data["items"] = pd.Series(enchants_permitted())
data.set_index("id_name", inplace=True)

In [6]:
roma_num = {"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5}

In [10]:
data["max_level"] = data["max_level"].apply(lambda x: roma_num[x])

In [11]:
data.head()

,name,max_level,description,items
id_name,,,,
aqua_affinity,Aqua Affinity,1,Speeds up how fast you can mine blocks underwater,helmet
bane_of_arthropods,Bane of Arthropods,5,Increases attack damage against arthropods,sword axe
blast_protection,Blast Protection,4,Reduces blast and explosion damage,armor
channeling,Channeling,1,Summons a lightning bolt at a targeted mob whe...,trident
binding_curse,Curse of Binding,1,Cursed item can not be removed from player,armor


In [5]:
data.to_csv(OUTFILE)